In [ ]:
#import datasetBuilder
import tests
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from importlib import reload
import pandas as pd
import datasetBuilder
import tools
import scipy
from sklearn.ensemble import HistGradientBoostingClassifier as hgbc
from sklearn.ensemble import GradientBoostingClassifier as gbc
from sklearn.ensemble import RandomForestClassifier as rfc
import sklearn.base
import pickle
import copy


In [ ]:
first_bases = np.load('C:\\Users\\jonah\\projects\\specsim_res\\first_bases.npy')
second_bases = np.load('C:\\Users\\jonah\\projects\\specsim_res\\second_bases.npy')
#first_target = target[np.isin(target['inchi_base'],first_bases)]
#second_target = target[np.isin(target['inchi_base'],second_bases)]


In [ ]:
reload(datasetBuilder)
reload(tools)
noise_threshes=[0.01,0.1,0.2]
centroid_tolerance_vals = [0.05,1]
centroid_tolerance_types=['da','ppm']
powers=[0.25,1,3,'ent']
sim_methods = ['lorentzian','max_entropy','squared_chord','lorentzian_jonah','rbf','ave_bhattacharya_2','max_bhattacharya_2']

matches = datasetBuilder.create_matches_df(first_target,10,100,1e6)
og_matches = matches.copy()
datasetBuilder.add_noises_to_matches(matches, scale_ratio=0.5, mult=200)

print(f'match len predrop {len(matches)}')
matches=matches.dropna(how='any')
print(f'match len postdrop {len(matches)}')

cleaned, mod = datasetBuilder.create_model_dataset(matches, sim_methods = sim_methods, noise_threshes=noise_threshes, centroid_tolerance_vals=centroid_tolerance_vals, centroid_tolerance_types=centroid_tolerance_types,powers=powers)
mod.to_csv('C:\\Users\\jonah\\projects\\specsim_res\\noise_first_0.5_200.csv')

In [ ]:
del(matches)
del(mod)
del (first_target)
second_target = target[np.isin(target['inchi_base'],second_bases)]

noise_threshes=[0.01,0.1,0.2]
centroid_tolerance_vals = [0.05,1]
centroid_tolerance_types=['da','ppm']
powers=[0.25,1,3,'ent']
sim_methods = ['lorentzian','max_entropy','squared_chord','lorentzian_jonah','rbf','ave_bhattacharya_2','max_bhattacharya_2']

matches = datasetBuilder.create_matches_df(second_target,10,100,1e6)
datasetBuilder.add_noises_to_matches(matches, scale_ratio=0.5, mult=200)
print(f'match len predrop {len(matches)}')
matches=matches.dropna(how='any')
print(f'match len postdrop {len(matches)}')

mod = datasetBuilder.create_model_dataset(matches, sim_methods = sim_methods, noise_threshes=noise_threshes, centroid_tolerance_vals=centroid_tolerance_vals, centroid_tolerance_types=centroid_tolerance_types,powers=powers)
mod.to_csv('C:\\Users\\jonah\\projects\\specsim_res\\noise_second_0.5_200.csv')
del(matches)
del(mod)

In [ ]:
def generate_keep_indices(noise_threshes, centroid_tolerance_vals, powers, spec_features, spec_change_features, sim_methods, any_=False, nonspecs=False, init_spec=False):

    if nonspecs:
        keep_indices= list(range(14))
    else:
        keep_indices=list()

    if init_spec:
        keep_indices+=list(range(14,24))

    ind=24
    for i in noise_threshes:
        for j in centroid_tolerance_vals:
            for k in powers:
                
                for l in spec_features:
                    if any_:
                        if True in [i,j,k,l]:
                            keep_indices.append(ind)
                    else:
                        if i==j==k==l==True:
                            keep_indices.append(ind)
                    ind+=1

                for l in spec_change_features:
                    if any_:
                        if True in [i,j,k,l]:
                            keep_indices.append(ind)
                    else:
                        if i==j==k==l==True:
                            keep_indices.append(ind)
                    ind+=1


                for l in sim_methods:
                    
                    if any_:
                        if True in [i,j,k,l]:
                            keep_indices.append(ind)
                    else:
                        if i==j==k==l==True:
                            keep_indices.append(ind)
                    ind+=1

    return keep_indices


In [ ]:
train.columns[14:24]

In [ ]:
noise_threshes=[0.01,0.1,0.2]
centroid_tolerance_vals = [0.05,1]
centroid_tolerance_types=['da','ppm']
powers=[0.25,1,3,'ent']
sim_methods = ['lorentzian','max_entropy','squared_chord','lorentzian_jonah','rbf','ave_bhattacharya_2','max_bhattacharya_2']


noise_threshes=[True,True,True]
centroid_tolerance_values=[True,False]
powers=[True,True,True,True]
spec_features=[True,True, False,False,False,False,False,False]#8
spec_change_features = [False for i in range(8)]#8
sim_methods=[True,False,False,False,False,False]#7

#cols['nonspec_only']=generate_keep_indices(noise_threshes, centroid_tolerance_values, powers, spec_features, spec_change_features, sim_methods, any_=True, nonspecs=True)
cols2['first_2_spec_lor_only']=generate_keep_indices(noise_threshes, centroid_tolerance_values, powers, spec_features, spec_change_features, sim_methods, any_=False, nonspecs=False, init_spec=True)



In [ ]:
with open('/Users/jonahpoczobutt/projects/specsim_res/col_dict.pickle', 'rb') as handle:
    cols = pickle.load(handle)

In [ ]:
cols_auc_test = [
    ('all',cols['all'],hgbc()),
    ('all_but_nonspec',cols['all_but_nonspec'],hgbc()),
    ('nonspec_only',cols['nonspec_only'],hgbc()),
    ('first_2_spec_lor_only',cols2['first_2_spec_lor_only'],hgbc()),
    ('first_2_spec',cols2['first_2_spec'],hgbc())
]

for i in cols_auc_test:
    i[2].fit(pd.concat((train.iloc[:,i[1]],val.iloc[:,i[1]])), pd.concat((train.iloc[:,-1:],val.iloc[:,-1:])))

    print(f'trained: {i[0]}')



In [ ]:
metrics=list()

for i in cols['all_sims']:

    metrics.append((test.columns[i], i))
    

In [ ]:
reload(tests)

aucs_res = tests.run_metrics_models_auc(metrics, cols_auc_test, test)

In [ ]:
aucs_res = {k: v for k, v in sorted(aucs_res.items(), key=lambda item: -1*item[1])}

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

a=np.random.beta(a=1,b=5,size=10000)*200
plt.hist(a, weights=np.ones(len(a)) / len(a), bins=40)
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.title('Distribution Over Peak Heights When Y=100')
plt.show()

In [ ]:
aucs_res

In [ ]:
aucs_res2

In [ ]:
from collections import Counter
mods = list()

for i in res_dict:

    mods=mods+[j[0] for j in res_dict[i]]

for i in res_dict2:

    mods=mods+[j[0] for j in res_dict2[i]]

Counter(mods)

In [ ]:
train = pd.read_csv('/Volumes/Untitled/specsim_res/noise_first_1_100.csv').iloc[:,1:]
test = pd.read_csv('/Volumes/Untitled/specsim_res/noise_second_1_100.csv').iloc[:,1:]

print(len(train))
train.dropna(how='any', inplace=True)
print(len(train))

print(len(test))
test.dropna(how='any', inplace=True)
print(len(test))

val=test.iloc[:int(2e5)]
test=test.iloc[int(2e5):]



In [ ]:
train.iloc[:10,cols['nonspec_only']]

In [ ]:
with open('/Users/jonahpoczobutt/projects/specsim_res/col_dict.pickle', 'rb') as handle:
    cols = pickle.load(handle)

cols['random_half']=list(np.random.choice(576,size=int(576/2),replace=False))
cols['random_quarter']=list(np.random.choice(576,size=int(576/4),replace=False))
cols['random_eighth']=list(np.random.choice(576,size=int(576/8),replace=False))
cols['random_tenth']=list(np.random.choice(576,size=int(576/10),replace=False))
cols['all']=list(range(576))

print(len(cols))
for i in cols:
    print(f'{i} {len(cols[i])}')

In [ ]:


def zero_one_loss(preds, true):
    
    preds=np.array(preds).squeeze()
    true=np.array(true).squeeze()
    
    return sum(abs(true-preds))/len(true)

def best_model_select(models, train, val, test):
    
    model_errs = np.zeros(len(models))
    train['match']=train['match'].astype(int)
    val['match']=val['match'].astype(int)
    test['match']=test['match'].astype(int)

    best_model = None
    best_error=np.inf

    i=0
    for model in models:

        clf = sklearn.base.clone(model)
        clf.fit(train.iloc[:,:-1], train.iloc[:,-1])
        val_error = zero_one_loss(clf.predict(val.iloc[:,:-1]),val.iloc[:,-1:].to_numpy())

        if val_error < best_error:
            best_model=copy.deepcopy(clf)
            best_error = val_error

        del(clf)
        model_errs[i]=val_error
        i+=1

    return str(best_model), zero_one_loss(best_model.predict(test.iloc[:,:-1]),test.iloc[:,-1:].to_numpy()), model_errs

def best_models_by_subset(cols, train_sizes, models, train, val, test):

    res_dict=dict()
    for key, value in cols.items():
        res_dict[key]=list()

        for size in train_sizes:
    
            res_dict[key].append(best_model_select(models, train.iloc[:size,value+[576]], val.iloc[:,value+[576]], test.iloc[:,value+[576]]))

            print(f'finished {key} for {size}')

    return res_dict



In [ ]:

models = [
        hgbc(),
        hgbc(learning_rate=0.01),
        hgbc(learning_rate=0.5),
        hgbc(max_iter=200),
        hgbc(learning_rate=0.01, max_iter=200),
        hgbc(learning_rate=0.5, max_iter=200),
        hgbc(min_samples_leaf=10),
        hgbc(learning_rate=0.01,min_samples_leaf=10),
        hgbc(learning_rate=0.5,min_samples_leaf=10),
        hgbc(max_iter=200,min_samples_leaf=10),
        hgbc(learning_rate=0.01, max_iter=200,min_samples_leaf=10),
        hgbc(learning_rate=0.5, max_iter=200,min_samples_leaf=10),
        ]

#model, test_err, val_errs = best_model_select(models, train[:1000], val[:1000], test[:1000]) 

In [ ]:
train_sizes=[int(8e5)]
res_dict2 = best_models_by_subset(cols2, train_sizes=train_sizes, models=models, train=train, val=val, test=test )
        



In [ ]:
with open('/Users/jonahpoczobutt/projects/specsim_res/res_dict2.pickle', 'wb') as handle:
    pickle.dump(res_dict2, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
res_dict2

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(10,6))
for i in res_dict2:

    if i in res_dict2:
    #if 'all' in i or i=='nonspec_only':
        test_errors = [j[1] for j in res_dict2[i]]
        plt.scatter(train_sizes, test_errors, label=i)


plt.title('Zero One Error by Proportion of Features Included')
plt.xlabel('# of Training Points')
plt.ylabel('Zero One Error')
plt.legend(loc='best')
plt.show()




In [ ]:
import spectral_similarity

sims=list()
for i in spectral_similarity.methods_range:
    sims.append(i)
    sims.append('max_'+i)
    sims.append('min_'+i)
    sims.append('ave_'+i)
    
mod_data = datasetBuilder.create_model_dataset(target, 10, sims, 10, nonspec_features=False, spec_features=False)

In [ ]:
mod_data=mod_data.iloc[:,1:]
mod_data.columns=sims
mod_data

In [ ]:
import scipy.stats
scipy.stats.entropy([1 for i in range(10000)])

In [ ]:
import numpy as np
import plotAndOrderResults

In [ ]:
import pandas as pd
df_res=pd.read_csv("C:\\Users\\jonah\\projects\\SpecSim\\allsim_res_4.csv")

In [ ]:
plotAndOrderResults.add_evals_to_df(df_res)

In [ ]:
plotAndOrderResults.order_criterion_from_df(df_res)

In [ ]:
plotAndOrderResults.order_criterion_from_df(df_res)

In [ ]:
yool = spectral_similarity.multiple_similarity(b,a, ms2_da=0.05, methods=['max_lorentzian','reverse_lorentzian'])

In [ ]:
yool['reverse_lorentzian']

In [ ]:
a

In [ ]:
yool['reverse_entropy']

In [ ]:
match = tools.match_peaks_in_spectra(a,b, ms2_da=0.05)

In [ ]:
import math_distance

In [ ]:
math_distance._select_common_peaks(a,b)

In [ ]:
yool = a>0

In [ ]:
yool